In [14]:
!pip install transformers torch
!pip install python-dotenv

In [15]:
!pip install langchain langchain-openai openai faiss-cpu rank-bm25 pymupdf numpy
!pip install -U langchain-community
!pip install deepeval
!pip install langchain_openai
!pip install langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 71.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchai

In [16]:
# Create the project directory and subfolders
!mkdir -p /kaggle/working/project_root/evaluation

In [17]:
import sys
import os

In [18]:
# Add project_root and its subdirectories to Python path
sys.path.append('/kaggle/working/project_root')
sys.path.append('/kaggle/working/project_root/evaluation')

In [19]:
# Move helper_functions.py to project_root
!cp /kaggle/input/helper-functions-zip/helper_functions.py /kaggle/working/project_root/

# Move evalute_rag.py to project_root/evaluation
!cp /kaggle/input/evaluate-rag-zip/evalute_rag.py /kaggle/working/project_root/evaluation/


In [20]:
import os
import sys
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

In [21]:
# Load environment variables from a .env file
load_dotenv()

False

In [22]:
# Initialize the SentenceTransformer model
embedding_model_name = os.getenv('EMBEDDING_MODEL', 'all-MiniLM-L6-v2')  # Default model
embedding_model = SentenceTransformer(embedding_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# Add the parent directory to the path since we work with notebooks
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [30]:
!pip install --upgrade langchain-core pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3


In [31]:
# Import custom helper functions
from helper_functions import *
# from evaluation.evalute_rag import *

In [32]:
path = "/kaggle/input/understanding-climate-change-pdf/Understanding_Climate_Change.pdf"

In [33]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [34]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Encodes a PDF book into a vector store using Hugging Face transformer embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.
        model_name: The Hugging Face model to use (default: "sentence-transformers/all-MiniLM-L6-v2").

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    chunks = text_splitter.split_documents(documents)

    # Extract text from the chunked documents
    chunk_texts = [chunk.page_content for chunk in chunks]

    # Initialize the HuggingFaceEmbeddings with the model name (no need to load the model manually)
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # Create a FAISS vector store from the chunk texts and the embeddings
    vectorstore = FAISS.from_texts(chunk_texts, embeddings)

    return vectorstore

In [35]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})


In [36]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate. 
Fossil Fuels 
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today. 
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch,